In [21]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [22]:
!pip install transformers beautifulsoup4 requests pandas numpy

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [24]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [32]:
tokens = tokenizer.encode('WOW IT IS NICE, BUT I DON\'T LIKE IT)',return_tensors='pt')
tokens

tensor([[  101, 94608, 10197, 10127, 24242,   117, 10502,   151, 11530,   112,
           162, 11531, 10197,   114,   102]])

In [33]:
result = model(tokens)

In [27]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7668,  0.6999,  3.7177,  1.2586, -2.5858]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
int(torch.argmax(result.logits))+1

3

In [48]:
r = requests.get("https://www.yelp.ca/biz/tamasha-toronto")
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [49]:
reviews

["A fun time! This is an indian food/ party spot just below Davisville. Food was fairly good if nothing special. Garlic naan was fresh and the kali mirch had a decent kick to it. The nachos with paneer were surprisingly good. The real reason to come though is to have a late night drink and a good time. Ball don't lie: when we went a few weeks ago Tamasha was by far the busiest restaurant in the neighbourhood. Service was incredibly friendly. We'll be back sometime.",
 "The menu is quite basic. Super overpriced.Had the chicken tandoori momos. \xa0They were not fried or boiled, but over roasted and stiff. \xa0Its flavoured on the outside but the chicken doesn't have the tandoori seasoning. \xa0They provide plain mayonnaise with it. \xa0A thick mint-cilantro raita would have made more sense. 8 momos for $18.07 was not worth what was received. \xa0$8 would have been more reasonable..Came in the middle of the week at lunch time. \xa0It was dead, so lots of seating.It is not Muslim owned but

In [50]:
import pandas as pd
import numpy as np

In [62]:
df = pd.DataFrame(np.array(reviews),columns=['review'])
df['review'].iloc[2]

'So far, I have tried their lunch special. The Chicken Tikka Kathi Roll was delicious and good value for money. Lamb Rogan Josh - Tastes good but the problem is I did not realize how much rice would come with it. Price a bit steep for what you actually get. Food could be spicier but I guess they have to cater to all palates.Both times the order was ready in the time they said it would be. Service is friendly enough for take-out. I see lots of people there on the weekends....so that is probably a good sign. Will try the dinner menu soon.'

In [65]:
def sentiment_score(reviews):
    tokens = tokenizer.encode(reviews,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [66]:
sentiment_score(df['review'].iloc[1])

2

In [67]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [68]:
df

,review,sentiment
0,A fun time! This is an indian food/ party spot...,4
1,The menu is quite basic. Super overpriced.Had ...,2
2,"So far, I have tried their lunch special. The ...",3
3,The food is super delicious and the portions a...,3
4,"Experience/ambience: not great, could be bette...",3
5,I came here with a friend of mine to get some ...,3
6,"I visited this lounge yesterday, everything wa...",1
7,It was a pleasure visiting this restaurant. G...,5
8,Great atmosphere & food! Slow service. Expensi...,2
9,food is and manager is soo greedy dnt go over...,2


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df, *['sentiment'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df, *['sentiment'], **{})
chart